**Note:**

It's a good practise to "restart the session", whenever changes are made to the github repository


##**Step 1:** GitHub setup

1) Clone/Update the projects github repository

In [1]:
# Use for the first time to clone the github repo or when the repo is updated
%rm -rf /content/MarketNeutral_Trading_multiple_pairs/
%cd /content
!git clone https://github.com/WQU-Capstone-11205/MarketNeutral_Trading_multiple_pairs.git
%cd /content/MarketNeutral_Trading_multiple_pairs/

/content
Cloning into 'MarketNeutral_Trading_multiple_pairs'...
remote: Enumerating objects: 293, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 293 (delta 56), reused 21 (delta 21), pack-reused 208 (from 1)
Receiving objects: 100% (293/293), 1.74 MiB | 13.72 MiB/s, done.
Resolving deltas: 100% (141/141), done.
/content/MarketNeutral_Trading_multiple_pairs


2) Add the project's github repository's path to the system path

In [2]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('/content/MarketNeutral_Trading_multiple_pairs')

3) Install projects required packages

In [3]:
!pip install -r requirements.txt

##**Step 2:** Data loading

- Load data and convert to distance spread

In [4]:
from util.ff_benchmark import get_ff_benchmark_returns
from data_loading.fetch_data import fetch_from_yfinance
from data_loading.PairsSpread import SP500PairSpread

start_date = '2015-01-01' #'2019-01-01' # '2005-01-01'
end_date = '2025-01-01' # '2024-01-01' # '2025-01-01'
in_sample_cutoff_date = '2022-01-01' # '2017-01-01' # After this date the cointegration fails

selector = SP500PairSpread(
    selection_start=start_date,
    selection_end=end_date,
    method="distance",
    pairs_per_sector=4
)

spread = selector.distance_spread()

# price data used to compute cointegration
tickers = sorted(set(t for pair in spread.columns for t in pair.split("-")))
price_df = fetch_from_yfinance(tickers, start_date, end_date)

Fetching SP500 pairs from Wikipedia...
Calculating spread...


[*********************100%***********************]  74 of 74 completed
[*********************100%***********************]  74 of 74 completed


**Pair characteristics:**

In [5]:
from util.seed_random import seed_random

seed_random(42, device="cpu")

train_spread = spread.loc[:in_sample_cutoff_date]
test_spread = spread.loc[in_sample_cutoff_date:]

spread_returns = spread.pct_change().dropna()
train_spread_returns = spread_returns.loc[:in_sample_cutoff_date]
test_spread_returns = spread_returns.loc[in_sample_cutoff_date:]

Run Ablations:

In [6]:
from train.train_loop_rl import train_loop_rl
from backtest.evaluate_loop_rl import evaluate_loop_rl

bocpd_params = {'hazard': 100, 'mu': 2, 'kappa': 10.0, 'alpha': 10.0, 'beta': 0.8}
vae_params = {'input_dim': 2, 'latent_dim': 12, 'hidden_dim': 64, 'lr': 0.0001, 'vae_seq_len': 1, 'kl_wt': 0.0005}
rl_params = {'state_dim': 12, 'action_dim': 1, 'hidden_dim': 128, 'actor_l2': 0.0001, 'lr': 5e-06, 'gamma': 0.95, 'action_l2': 0.001, 'cp_weight': 0.15, 'var_penalty': 1e-05, 'var_window': 5, 'dd_penalty': 0.5, 'dd_threshold': 0.5, 'actor_lr': 0.001, 'tau': 0.001}
joint_params = {'state_window': 25, 'base_action_sigma': 0.01, 'wt_multplier': 1.8, 'buffer_size_updates': 256, 'sample_batch_size': 128, 'transaction_cost': 0.001, 'tc_scale': 0.8, 'exploration_alpha': 2.0, 'update_every': 10}


configs = {
    "RL_only": dict(use_bocpd=False, use_vae=False),
    "RL_BOCPD": dict(use_bocpd=True, use_vae=False),
    "RL_VAE": dict(use_bocpd=False, use_vae=True),
    "FULL": dict(use_bocpd=True, use_vae=True)
}

results = {}

for name, cfg in configs.items():
    print(f"\nRunning {name}")

    train_loop_rl(
        train_spread,
        bocpd_params,
        vae_params,
        rl_params,
        joint_params,
        use_bocpd=cfg["use_bocpd"],
        use_vae=cfg["use_vae"],
        save_dir=f"checkpoints_{name}"
    )

    metrics, _ = evaluate_loop_rl(
        test_spread,
        bocpd_params,
        vae_params,
        rl_params,
        joint_params,
        use_bocpd=cfg["use_bocpd"],
        use_vae=cfg["use_vae"],
        load_dir=f"checkpoints_{name}"
    )

    results[name] = metrics["sharpe_ratio"]



Running RL_only
Epoch 000 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 0.681 | Cumulative PnL = 696.169
Saved all models + optimizers
Saved best models at epoch 000 (Sharpe=0.148)
Epoch 001 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 2.143 | Cumulative PnL = 557.493
No improvement. Early stopping patience counter = 1/5
Epoch 002 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 3.981 | Cumulative PnL = 301.347
No improvement. Early stopping patience counter = 2/5
Epoch 003 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 5.867 | Cumulative PnL = 714.867
No improvement. Early stopping patience counter = 3/5
Epoch 004 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 7.039 | Cumulative PnL = 715.222
No improvement. Early stopping patience counter = 4/5
Epoch 005 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 7.160 | Cumulative PnL = 715.256
No improvement. Early stopping patience counter = 5/5
EARLY STOPPING TRIGGERED at epoch 5
Multi-pair RL

In [8]:
print("Results of Ablations of Hybrid model:")
for res in results:
  print(f"{res}: {round(results[res],5)}")

Results of Ablations of Hybrid model:
RL_only: 0.05415
RL_BOCPD: 0.05459
RL_VAE: 0.05401
FULL: 0.05446


In [9]:
from train.train_loop_rl import train_loop_rl
from backtest.evaluate_loop_rl import evaluate_loop_rl

bocpd_params = {'hazard': 100, 'mu': 2, 'kappa': 10.0, 'alpha': 10.0, 'beta': 0.8}
vae_params = {'input_dim': 2, 'latent_dim': 12, 'hidden_dim': 64, 'lr': 0.001, 'vae_seq_len': 1, 'kl_wt': 0.0005}
rl_params = {'state_dim': 12, 'action_dim': 1, 'hidden_dim': 128, 'actor_l2': 0.0001, 'lr': 1e-04, 'gamma': 0.95, 'action_l2': 0.001, 'cp_weight': 0.15, 'var_penalty': 1e-05, 'var_window': 5, 'dd_penalty': 0.5, 'dd_threshold': 0.5, 'actor_lr': 0.001, 'tau': 0.001}
joint_params = {'state_window': 25, 'base_action_sigma': 0.01, 'wt_multplier': 1.8, 'buffer_size_updates': 256, 'sample_batch_size': 128, 'transaction_cost': 0.001, 'tc_scale': 0.8, 'exploration_alpha': 2.0, 'update_every': 10}


configs = {
    "RL_only": dict(use_bocpd=False, use_vae=False),
    "RL_BOCPD": dict(use_bocpd=True, use_vae=False),
    "RL_VAE": dict(use_bocpd=False, use_vae=True),
    "FULL": dict(use_bocpd=True, use_vae=True)
}

results = {}

for name, cfg in configs.items():
    print(f"\nRunning {name}")

    train_loop_rl(
        train_spread,
        bocpd_params,
        vae_params,
        rl_params,
        joint_params,
        use_bocpd=cfg["use_bocpd"],
        use_vae=cfg["use_vae"],
        save_dir=f"checkpoints_{name}"
    )

    metrics, _ = evaluate_loop_rl(
        test_spread,
        bocpd_params,
        vae_params,
        rl_params,
        joint_params,
        use_bocpd=cfg["use_bocpd"],
        use_vae=cfg["use_vae"],
        load_dir=f"checkpoints_{name}"
    )

    results[name] = metrics["sharpe_ratio"]



Running RL_only
Epoch 000 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 2.799 | Cumulative PnL = 373.112
Saved all models + optimizers
Saved best models at epoch 000 (Sharpe=-1.580)
Epoch 001 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 10.030 | Cumulative PnL = 959.541
Saved all models + optimizers
Saved best models at epoch 001 (Sharpe=0.572)
Epoch 002 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 12.737 | Cumulative PnL = 715.473
No improvement. Early stopping patience counter = 1/5
Epoch 003 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 12.615 | Cumulative PnL = 715.449
No improvement. Early stopping patience counter = 2/5
Epoch 004 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 11.917 | Cumulative PnL = 715.439
No improvement. Early stopping patience counter = 3/5
Epoch 005 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 11.412 | Cumulative PnL = 715.440
No improvement. Early stopping patience counter = 4/5
Epoch 006 | recon los

In [10]:
print("Results of Ablations of Hybrid model:")
for res in results:
  print(f"{res}: {round(results[res],5)}")

Results of Ablations of Hybrid model:
RL_only: 0.05454
RL_BOCPD: 0.05443
RL_VAE: 0.14089
FULL: 0.04785


In [6]:
from train.train_loop_rl import train_loop_rl
from backtest.evaluate_loop_rl import evaluate_loop_rl

bocpd_params = {'hazard': 100, 'mu': 2, 'kappa': 10.0, 'alpha': 10.0, 'beta': 0.8}
vae_params = {'input_dim': 2, 'latent_dim': 12, 'hidden_dim': 64, 'lr': 0.01, 'vae_seq_len': 1, 'kl_wt': 0.0005}
rl_params = {'state_dim': 12, 'action_dim': 1, 'hidden_dim': 128, 'actor_l2': 0.0001, 'lr': 1e-04, 'gamma': 0.95, 'action_l2': 0.001, 'cp_weight': 0.15, 'var_penalty': 1e-05, 'var_window': 5, 'dd_penalty': 0.5, 'dd_threshold': 0.5, 'actor_lr': 0.001, 'tau': 0.001}
joint_params = {'state_window': 25, 'base_action_sigma': 0.01, 'wt_multplier': 1.8, 'buffer_size_updates': 256, 'sample_batch_size': 128, 'transaction_cost': 0.001, 'tc_scale': 0.8, 'exploration_alpha': 2.0, 'update_every': 10}


configs = {
    "RL_only": dict(use_bocpd=False, use_vae=False),
    "RL_BOCPD": dict(use_bocpd=True, use_vae=False),
    "RL_VAE": dict(use_bocpd=False, use_vae=True),
    "FULL": dict(use_bocpd=True, use_vae=True)
}

results = {}

for name, cfg in configs.items():
    print(f"\nRunning {name}")

    train_loop_rl(
        train_spread,
        bocpd_params,
        vae_params,
        rl_params,
        joint_params,
        use_bocpd=cfg["use_bocpd"],
        use_vae=cfg["use_vae"],
        save_dir=f"checkpoints_{name}"
    )

    metrics, _ = evaluate_loop_rl(
        test_spread,
        bocpd_params,
        vae_params,
        rl_params,
        joint_params,
        use_bocpd=cfg["use_bocpd"],
        use_vae=cfg["use_vae"],
        load_dir=f"checkpoints_{name}"
    )

    results[name] = metrics["sharpe_ratio"]



Running RL_only
Epoch 000 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 2.807 | Cumulative PnL = 496.368
Saved all models + optimizers
Saved best models at epoch 000 (Sharpe=-1.347)
Epoch 001 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 11.118 | Cumulative PnL = 741.275
Saved all models + optimizers
Saved best models at epoch 001 (Sharpe=0.160)
Epoch 002 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 17.085 | Cumulative PnL = 559.254
No improvement. Early stopping patience counter = 1/5
Epoch 003 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 18.363 | Cumulative PnL = 472.777
No improvement. Early stopping patience counter = 2/5
Epoch 004 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 18.617 | Cumulative PnL = 273.203
No improvement. Early stopping patience counter = 3/5
Epoch 005 | recon loss = 0.000 | kl loss = 0.000 | policy loss = 21.705 | Cumulative PnL = 536.912
No improvement. Early stopping patience counter = 4/5
Epoch 006 | recon los

In [7]:
print("Results of Ablations of Hybrid model:")
for res in results:
  print(f"{res}: {round(results[res],5)}")

Results of Ablations of Hybrid model:
RL_only: 3.47164
RL_BOCPD: 0.05462
RL_VAE: 0.0547
FULL: 0.05253
